In [15]:
# day 32
# RNN 
from keras.models import Sequential # 케라스 층생성 모듈
import numpy as np


In [16]:
# model=Sequential()
# # model.add(Embedding(voc, hidden_size, input_length)) # 기본적 케라스 작업방법
# # 일반적 계층 추가

# # model.add(Dense(2, input_dim=4, init='uniform', activation='softmax')) 
# # 자연어처리 voca 생성 당시, 1000개..input_dim = 1000),init='normal') 가중치 초기화법
# # 다중분류 - 출력층 softmax
# # 이진분류 - sigmoid
# # linear (default value)

# model.add(Dense(4, input_dim=10, init='normal', activation = 'softmax'))
# # 출력이 4개 입력데이터 차원 10 
# model.add(Dense(1, activation='sigmoid'))
# model.summary() # : 요약정보
# model.compile() # : 함수설정
# model.fit() # : 학습
# model.evaluate() # : 모델 평가
# model.predict() # : 예측
# model.save()
# model.load()



In [17]:
import tensorflow as tf
tf.__version__ # tf의 버젼이 나온다
tf.get_default_graph() # 기본그래프정보 출력

In [18]:
# 내가 직접 그래프 만들어 거기에다가 노드를 지정해주자 ! 
testGraph = tf.Graph() # 그래프 클래스 객체 생성 
with testGraph.as_default(): # 사용자 그래프에 노드정의 
    x = tf.constant(10)
    y = x + 20
# testGraph


In [19]:
# 그래프 실행을 위해서 세션을 생성해줘야한다 
# Session() : 세션객체 생성, 분산처리 환경 => 통신을 연결하겠다
# 노드를 실행할수 있는 연결객체를 생성하겠다 
# run() : (그래프 구성 노드) 기입시 연산을 처리한다 => 결과를 반환
# ex) sess.run() 
# close() : session 종료문

x = tf.constant(5)
y = x + 10
sess=tf.Session()
print(sess.run(x))
print(sess.run(y))
sess.close()


5
15


In [20]:
# with 구문은 close() 문을 달아주지않아도 자동적으로 처리해준다
with tf.Session() as sess:
    print(sess.run(x))
    # 위세션이 사용하는 그래프는 default 기본그래프를 사용한다
    # 특별히 그래프를 지정하지 않았으므로, 기본그래프로 세션실행하겠다 구문 
    

5


In [21]:
with tf.Session(graph=testGraph) as sess:
    print(sess.run(x))


ValueError: Fetch argument <tf.Tensor 'Const_1:0' shape=() dtype=int32> cannot be interpreted as a Tensor. (Tensor Tensor("Const_1:0", shape=(), dtype=int32) is not an element of this graph.)

In [22]:
# tensorflow.session() 
# sess.run # 노드 실행 
# ... # 매번 하기 불편하므로 

# sess=tf.InteractiveSession()
# x.eval()
# y.eval()

In [23]:
# scope() 
# name space (이름공간)
# 폴더 (file system) 개념과 유사하다 
# 같은 폴더 내에서 같은이름은 안돼 

with tf.name_scope('ns1'):
    v3 = tf.Variable(1, name='var3')
    v4 = tf.add(v3, 2, name='var4')
print(v3.op.name)
    
    

ns1/var3


In [24]:
var1 = tf.Variable(5,name='var1') 
var2 = tf.add(var1, 5, name='var2') 
var1.op.name

'var1'

In [25]:
## PPT 
# 입력 4 # 출력 2 (hidden s[0])
# 'hihello'
# 이정도면 time-Step : 6, input_dim : 5, hideensize : 5
idx2char = [ 'h','i','e','l','o']
xdata = [[0,1,0,2,3,3]]
x_onehot =[[
    [1,0,0,0,0],
    [0,1,0,0,0],
    [1,0,0,0,0],
    [0,0,1,0,0],
    [0,0,0,1,0],
    [0,0,0,1,0]]]
ydata = [[1,0,2,3,3,4]] 
tf.reset_default_graph()
num_classes = 5
input_dim = 5 # 입력데이터의 차원(단어벡터의길이)
hidden_size = 5
batch_size = 1
sequence_length = 6 # hihell 을 학습시키는것
learning_rate = 0.05

x = tf.placeholder(tf.float32, [None, sequence_length, input_dim])
y = tf.placeholder(tf.int32, [None, sequence_length])

cell = tf.contrib.rnn.BasicLSTMCell(num_units = hidden_size) 
# 출력되어지는 hidden size : 5 // 출력값의 갯수 ()
ini_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, x, initial_state = ini_state, 
                                     dtype = tf.float32)
x_fc = tf.reshape(outputs, [-1, hidden_size]) # -1 : None(전체)
outputs = tf.contrib.layers.fully_connected(inputs=x_fc, 
                                  num_outputs=num_classes)
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])
                            # [1,6,5]=>[[[0.1,0.2,0.5,0.1,0.1]],[],,,,]
weights = tf.ones([batch_size, sequence_length]) # weight 중요도 조절 # 동일한 중요도
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits = outputs, targets = y, 
                                                 weights = weights)
loss = tf.reduce_mean(sequence_loss)

prediction = tf.argmax(outputs, axis=2)

train = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)
with tf.Session() as sess : 
    sess.run(tf.global_variables_initializer())
    for i in range(101):
        lv, _= sess.run([loss, train], feed_dict ={x:x_onehot, y:ydata})
        res=sess.run(prediction,feed_dict ={x:x_onehot})
        print('step:',i,'cost:',lv,'prediction:', res, 'true Y:',ydata)
        resStr = [idx2char[c] for c in np.squeeze(res)]
        print('prediciton str:',''.join(resStr))
        
# num_units 는 출력개수 
# logits = score

# hihell
# ex) hihell - > ihello 
# hihell - > uihello 


step: 0 cost: 1.6111151 prediction: [[2 3 3 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
prediciton str: elllll
step: 1 cost: 1.5777253 prediction: [[3 3 3 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
prediciton str: llllll
step: 2 cost: 1.5502506 prediction: [[3 3 3 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
prediciton str: llllll
step: 3 cost: 1.5127641 prediction: [[3 3 3 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
prediciton str: llllll
step: 4 cost: 1.4621226 prediction: [[3 3 3 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
prediciton str: llllll
step: 5 cost: 1.4106606 prediction: [[3 3 3 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
prediciton str: llllll
step: 6 cost: 1.3585681 prediction: [[3 3 3 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
prediciton str: llllll
step: 7 cost: 1.3060474 prediction: [[3 3 3 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
prediciton str: llllll
step: 8 cost: 1.2498603 prediction: [[3 0 3 3 3 3]] true Y: [[1, 0, 2, 3, 3, 4]]
prediciton str: lhllll
step: 9 cost: 1.186063 prediction: [[2 0 3 3 3 3]] true Y: [[1, 

step: 96 cost: 0.00625436 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
prediciton str: ihello
step: 97 cost: 0.006168365 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
prediciton str: ihello
step: 98 cost: 0.006084885 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
prediciton str: ihello
step: 99 cost: 0.0060037836 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
prediciton str: ihello
step: 100 cost: 0.005925002 prediction: [[1 0 2 3 3 4]] true Y: [[1, 0, 2, 3, 3, 4]]
prediciton str: ihello


In [26]:
test = [[3,1,0,2,1]]
np.squeeze(test)
test2 = [idx2char [c] for c in np.squeeze(test)]
test2


['l', 'i', 'h', 'e', 'i']

In [27]:
import numpy as np
from tensorflow.contrib import rnn
import pprint
tf.reset_default_graph()
pp=pprint.PrettyPrinter(indent=4)
sess=tf.InteractiveSession()

# 

In [28]:
h = [ 1,0,0,0 ]
e = [ 0,1,0,0 ]
l = [ 0,0,1,0 ]
o = [ 0,0,0,1 ]


In [29]:
with tf.variable_scope('first_cell') as scope:
    hidden_size = 2 # hidden layer에 뉴런의 개수 => 추출하고자하는 특징 
    cell=tf.contrib.rnn.BasicRNNCell(num_units = hidden_size)
    print(cell.output_size, cell.state_size)
    xdata=np.array([[h,e,l,l,o],
                   [e,o,l,l,o],
                   [l,l,e,e,l]], dtype=np.float32)
    print(xdata.shape)
    # (1,5,4)
    # (batch_size, seq_len, input_dim)
    pp.pprint(xdata)
    outputs, _states = tf.nn.dynamic_rnn(cell, xdata, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
#     pp.pprint(_states.eval())


W0819 19:25:30.918316 11560 deprecation.py:323] From <ipython-input-29-f46551c574b8>:3: BasicRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.SimpleRNNCell, and will be replaced by that in Tensorflow 2.0.


2 2
(3, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[-0.30956295,  0.07987997],
        [-0.5629116 , -0.37921098],
        [ 0.69747996, -0.5360058 ],
        [ 0.4846841 ,  0.35727847],
        [-0.3506697 ,  0.32269892]],

       [[-0.5989028 , -0.16203645],
        [ 0.34105095, -0.5012389 ],
        [ 0.5578495 ,  0.09776378],
        [ 0.15488012,  0.30306643],
        [-0.21238388,  0.0728706 ]],

       [[ 0.39254382, -0.12941985],
        [ 0.3492495 ,  0.16502088],
        [-0.7270347 ,  0.12118661],
        [-0.477411  , -0.61814255],
        [ 0.75803185, -0.5004653 ]]], dtype=float32)


In [30]:
# M(시트), V(그래프), C(컨트롤러(막대)) 분리

with tf.variable_scope('lstm_cell') as scope:
    hidden_size = 2 # hidden layer에 뉴런의 개수 => 추출하고자하는 특징 
    cell=tf.contrib.rnn.BasicLSTMCell(num_units = hidden_size)
#     print(cell.output_size, cell.state_size)
    xdata=np.array([[h,e,l,l,o],
                   [e,o,l,l,o],
                   [l,l,e,e,l]], dtype=np.float32)
#     print(xdata.shape)
    # (1,5,4)
    # (batch_size, seq_len, input_dim)
    pp.pprint(xdata)
    outputs, _states = tf.nn.dynamic_rnn(cell, xdata, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
#     pp.pprint(_states.eval())

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[ 0.02017836, -0.03614286],
        [ 0.00775723, -0.14905676],
        [ 0.07618628, -0.11244795],
        [ 0.12646106, -0.12488471],
        [ 0.04141374, -0.0603175 ]],

       [[-0.00103223, -0.12522997],
        [-0.04559198,  0.00471454],
        [ 0.03758546, -0.03537206],
        [ 0.09918653, -0.06980477],
        [ 0.02328135,  0.02203378]],

       [[ 0.07236499, -0.04258591],
        [ 0.1245627 , -0.07785878],
        [ 0.06046247, -0.19048057],
        [ 0.03405369, -0.22185272],
        [ 0.10152323, -0.15049268]]], dtype=float32)


In [31]:
with tf.variable_scope('lstm_cell_2') as scope:
    hidden_size = 2 
    cell=tf.contrib.rnn.BasicLSTMCell(num_units = hidden_size)
    xdata=np.array([[h,e,l,l,o],
                   [e,o,l,l,o],
                   [l,l,e,e,l]], dtype=np.float32)
    pp.pprint(xdata)
    outputs, _states = tf.nn.dynamic_rnn(cell, xdata, sequence_length=[5,3,4],
                                         dtype=tf.float32)
    sess.run(tf.global_variables_initializer())  
    pp.pprint(outputs.eval())

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)


W0819 19:25:31.348167 11560 deprecation.py:323] From C:\Users\user\Anaconda3\lib\site-packages\tensorflow\python\ops\rnn.py:244: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


array([[[-2.41659973e-02, -1.13974325e-01],
        [-7.68289417e-02, -5.22176363e-03],
        [-7.87679330e-02, -1.08298130e-01],
        [-6.18757047e-02, -1.77251294e-01],
        [ 2.37103719e-02, -2.37350404e-01]],

       [[-7.09921792e-02,  1.27833262e-01],
        [-1.29905390e-02,  1.93023894e-04],
        [-2.45636627e-02, -9.49059650e-02],
        [ 0.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00]],

       [[-1.59896892e-02, -9.31037813e-02],
        [-1.40158376e-02, -1.57956660e-01],
        [-6.23002537e-02,  2.34910864e-02],
        [-1.26427367e-01,  1.31095245e-01],
        [ 0.00000000e+00,  0.00000000e+00]]], dtype=float32)


In [32]:
with tf.variable_scope('init_lstm_cell') as scope:
    hidden_size = 2 
    batch_size = 3
    cell=tf.contrib.rnn.BasicLSTMCell(num_units = hidden_size)
    xdata=np.array([[h,e,l,l,o],
                   [e,o,l,l,o],
                   [l,l,e,e,l]], dtype=np.float32)
    pp.pprint(xdata)
    init_state = cell.zero_state(batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, xdata, dtype=tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())
    
    # 데이터 100개 / 1에폭 : 100개 데이터를 학습 
    # batch_size : 20개, batch_length : 100/20 = 5번 
    # (20개 단위로 학습) forward..reduce.mean..backward...weight, bias updating(20)
    

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)
array([[[-0.02912452, -0.17994003],
        [-0.16620423, -0.01071649],
        [-0.00490315, -0.03298961],
        [ 0.08654272, -0.07412206],
        [ 0.05814182, -0.00786786]],

       [[-0.1391202 ,  0.13458526],
        [-0.0929196 ,  0.11940601],
        [ 0.02974492,  0.07538293],
        [ 0.10296037, -0.00029293],
        [ 0.0682527 ,  0.0343336 ]],

       [[ 0.0879749 , -0.05106742],
        [ 0.14681752, -0.09920508],
        [-0.00689052,  0.00718846],
        [-0.14373058,  0.14190026],
        [ 0.00261996,  0.06467292]]], dtype=float32)


In [33]:
batch_size = 3
seq_len=5
in_dim=3
xdata = np.arange(45).reshape(batch_size, seq_len, in_dim) # ohe되어져있는것
print(xdata)

[[[ 0  1  2]
  [ 3  4  5]
  [ 6  7  8]
  [ 9 10 11]
  [12 13 14]]

 [[15 16 17]
  [18 19 20]
  [21 22 23]
  [24 25 26]
  [27 28 29]]

 [[30 31 32]
  [33 34 35]
  [36 37 38]
  [39 40 41]
  [42 43 44]]]


In [34]:
with tf.variable_scope('multi_rnn_cell') as scope:
    
    cell=tf.contrib.rnn.BasicLSTMCell(num_units = 5)
    cell=tf.contrib.rnn.MultiRNNCell([cell]*3) # layer 3개
    
    outputs, _states = tf.nn.dynamic_rnn(cell, xdata)
    sess.run(tf.global_variables_initializer(dtype=np.float32))
    pp.print(outputs.eval())
    
#     xdata=np.array([[h,e,l,l,o],
#                    [e,o,l,l,o],
#                    [l,l,e,e,l]], dtype=np.float32)
#     pp.pprint(xdata)
#     init_state = cell.zero_state(batch_size, tf.float32)
#     outputs, _states = tf.nn.dynamic_rnn(cell, xdata, initial_s,dtype=tf.float32)
#     sess.run(tf.global_variables_initializer())
#     pp.pprint(outputs.eval())
    
    

W0819 19:25:32.432221 11560 deprecation.py:323] From <ipython-input-34-dfd9818523dc>:4: MultiRNNCell.__init__ (from tensorflow.python.ops.rnn_cell_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
W0819 19:25:32.434217 11560 rnn_cell_impl.py:1642] At least two cells provided to MultiRNNCell are the same object and will share weights.


ValueError: If there is no initial_state, you must give a dtype.

In [35]:
ydata=tf.constant([[1,1,1]])
prediction1 = tf.constant([ [[0.2, 0.7], [0.1, 0.9], [0.3, 0.7] ]])
prediction2 = tf.constant([ [[0.4, 0.6], [0.4, 0.6], [0.6, 0.4] ]])
loss1 = tf.contrib.seq2seq.sequence_loss(prediction1, ydata)
sess.run(tf.global_variables_initializer())
loss1.eval()

# 각각의 cost 에 대하여 reduce.mean을 구한다 
# RNN 내 cost 구하기  
hihell  

TypeError: sequence_loss() missing 1 required positional argument: 'weights'